In [2]:
# rainfall_models_adilabad.py
# Comparative Study: CNN-LSTM vs Transformer vs TCN
# Sequence length = 30 | Train years = 2019–2023 | Test year = 2024

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_auc_score
)
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# -------------------------------
# CONFIG
# -------------------------------
DATA_FILES = {
    2019: r"C:\Users\priya\rainfall_prediction\Rainfall_prediction\datasets\hyderabad_2019.csv",
    2020: r"C:\Users\priya\rainfall_prediction\Rainfall_prediction\datasets\hyderabad_2020.csv",
    2021: r"C:\Users\priya\rainfall_prediction\Rainfall_prediction\datasets\hyderabad_2021.csv",
    2022: r"C:\Users\priya\rainfall_prediction\Rainfall_prediction\datasets\hyderabad_2022.csv",
    2023: r"C:\Users\priya\rainfall_prediction\Rainfall_prediction\datasets\hyderabad_2023.csv",
    2024: r"C:\Users\priya\rainfall_prediction\Rainfall_prediction\datasets\hyderabad_2024.csv",
}

TRAIN_YEARS = [2019, 2020, 2021, 2022, 2023]
TEST_YEAR = 2024
SEQ_LENGTH = 30
EPOCHS = 100
BATCH_SIZE = 32
RANDOM_STATE = 42

# -------------------------------
# LOAD + FEATURE ENGINEERING
# -------------------------------
def load_and_prepare_data(file_map=DATA_FILES):
    dfs = []
    for year in sorted(file_map.keys()):
        df = pd.read_csv(file_map[year]).assign(YEAR=year)
        dfs.append(df)

    full_df = pd.concat(dfs, ignore_index=True)

    required = ['YEAR', 'MO', 'DY', 'PRECTOTCORR']
    for col in required:
        full_df[col] = pd.to_numeric(full_df[col], errors='coerce')
    full_df.dropna(subset=required, inplace=True)

    full_df["datetime"] = pd.to_datetime(dict(
        year=full_df['YEAR'], month=full_df['MO'], day=full_df['DY']
    ))
    full_df["DOY"] = full_df["datetime"].dt.dayofyear
    full_df["sin_DOY"] = np.sin(2*np.pi*full_df["DOY"]/365.25)
    full_df["cos_DOY"] = np.cos(2*np.pi*full_df["DOY"]/365.25)

    # Lag features
    for lag in [1,3,7,14,30]:
        full_df[f"PRECTOTCORR_lag{lag}"] = full_df["PRECTOTCORR"].shift(lag)

    # Sea Level Temp (Hyderabad elev = 542m)
    full_df["SLT"] = full_df["TS"] + (0.0065 * 542)

    full_df.dropna(inplace=True)

    features = [
        "SLT","SLP","T2M","TS","T2M_MAX","T2M_MIN",
        "RH2M","WS10M_MAX","WS10M_MIN",
        "sin_DOY","cos_DOY",
        "PRECTOTCORR_lag1","PRECTOTCORR_lag3","PRECTOTCORR_lag7",
        "PRECTOTCORR_lag14","PRECTOTCORR_lag30"
    ]

    full_df["Rain"] = (full_df["PRECTOTCORR"] > 0).astype(int)

    train_df = full_df[full_df["YEAR"].isin(TRAIN_YEARS)]
    test_df  = full_df[full_df["YEAR"] == TEST_YEAR]

    return train_df, test_df, features, "Rain"

# -------------------------------
# PREPROCESS: SCALE + SMOTE + SEQUENCES
# -------------------------------
def preprocess_and_balance(train_df, test_df, features, target):
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(train_df[features])
    X_test_scaled  = scaler.transform(test_df[features])

    sm = SMOTE(random_state=RANDOM_STATE)
    X_res, y_res = sm.fit_resample(X_train_scaled, train_df[target])

    train_bal = pd.DataFrame(X_res, columns=features)
    train_bal[target] = y_res

    test_proc = pd.DataFrame(X_test_scaled, columns=features, index=test_df.index)
    test_proc[target] = test_df[target]
    test_proc["DOY"] = test_df["DOY"]

    return train_bal, test_proc, scaler

def create_sequences(df, features, target, seq_len):
    X, y = [], []
    for i in range(len(df) - seq_len):
        X.append(df[features].iloc[i:i+seq_len].values)
        y.append(df[target].iloc[i+seq_len])
    return np.array(X), np.array(y)

# -------------------------------
# MODEL A — CNN-LSTM
# -------------------------------
def build_cnn_lstm(seq_len, feat):
    inp = Input(shape=(seq_len, feat))
    x = Conv1D(64, 3, padding="same", activation="relu")(inp)
    x = Conv1D(64, 3, padding="same", activation="relu")(x)
    x = MaxPooling1D(2)(x)
    x = LSTM(64)(x)
    x = Dense(64, activation="relu")(x)
    x = Dropout(0.3)(x)
    out = Dense(1, activation="sigmoid")(x)

    model = Model(inp, out)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-4),
        loss="binary_crossentropy",
        metrics=["accuracy", tf.keras.metrics.Precision(),
                 tf.keras.metrics.Recall(), tf.keras.metrics.AUC()]
    )
    return model

# -------------------------------
# MODEL B — Transformer
# -------------------------------
def build_transformer(seq_len, feat, d_model=128, heads=4, ff_dim=256):
    inp = Input(shape=(seq_len, feat))
    x = Dense(d_model)(inp)

    pos = tf.Variable(tf.random.normal([1, seq_len, d_model]), trainable=True)
    x = x + pos

    for _ in range(2):
        att = MultiHeadAttention(num_heads=heads, key_dim=d_model//heads)(x, x)
        x = LayerNormalization()(x + att)
        ff = Dense(ff_dim, activation="relu")(x)
        ff = Dense(d_model)(ff)
        x = LayerNormalization()(x + ff)

    x = GlobalAveragePooling1D()(x)
    x = Dense(128, activation="relu")(x)
    x = Dropout(0.3)(x)
    out = Dense(1, activation="sigmoid")(x)

    model = Model(inp, out)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-4),
        loss="binary_crossentropy",
        metrics=["accuracy", tf.keras.metrics.Precision(),
                 tf.keras.metrics.Recall(), tf.keras.metrics.AUC()]
    )
    return model

# -------------------------------
# MODEL C — TCN (Dilated Causal Convolutions)
# -------------------------------
def TCN_block(x, filters, kernel=3, dilation=1):
    # Causal convolution
    conv1 = Conv1D(filters, kernel, dilation_rate=dilation,
                   padding="causal", activation="relu")(x)
    conv2 = Conv1D(filters, kernel, dilation_rate=dilation,
                   padding="causal", activation="relu")(conv1)

    # Residual connection
    if x.shape[-1] != filters:
        res = Conv1D(filters, 1, padding="same")(x)
    else:
        res = x

    return Add()([res, conv2])


def build_tcn(seq_len, feat):
    inp = Input(shape=(seq_len, feat))

    x = TCN_block(inp, filters=64, dilation=1)
    x = TCN_block(x,   filters=64, dilation=2)
    x = TCN_block(x,   filters=64, dilation=4)

    x = GlobalAveragePooling1D()(x)
    x = Dense(64, activation="relu")(x)
    x = Dropout(0.3)(x)
    out = Dense(1, activation="sigmoid")(x)

    model = Model(inp, out)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-4),
        loss="binary_crossentropy",
        metrics=["accuracy", tf.keras.metrics.Precision(),
                 tf.keras.metrics.Recall(), tf.keras.metrics.AUC()]
    )
    return model

# -------------------------------
# EVALUATION
# -------------------------------
def evaluate_and_print(y_true, y_prob):
    y_pred = (y_prob > 0.5).astype(int)
    print("Accuracy:", accuracy_score(y_true, y_pred))
    print("Precision:", precision_score(y_true, y_pred))
    print("Recall:", recall_score(y_true, y_pred))
    print("F1:", f1_score(y_true, y_pred))
    print("ROC-AUC:", roc_auc_score(y_true, y_prob))
    print(confusion_matrix(y_true, y_pred))
    print(classification_report(y_true, y_pred))

def train_model(model_builder, name, seq_len):
    print(f"\n===== Running {name} =====")
    train_df, test_df, features, target = load_and_prepare_data()
    train_bal, test_proc, scaler = preprocess_and_balance(train_df, test_df, features, target)

    X_train, y_train = create_sequences(train_bal, features, target, seq_len)
    X_test,  y_test  = create_sequences(test_proc, features, target, seq_len)

    model = model_builder(seq_len, len(features))

    cbs = [
        EarlyStopping(patience=10, restore_best_weights=True),
        ReduceLROnPlateau(patience=4, factor=0.5)
    ]

    model.fit(X_train, y_train,
              epochs=EPOCHS, batch_size=BATCH_SIZE,
              validation_split=0.2, callbacks=cbs, verbose=1)

    y_prob = model.predict(X_test).ravel()
    evaluate_and_print(y_test, y_prob)

    return model

# -------------------------------
# MAIN — Run All 3 Models
# -------------------------------
if __name__ == "__main__":
    train_model(build_cnn_lstm, "CNN-LSTM", SEQ_LENGTH)
    train_model(build_transformer, "Transformer", SEQ_LENGTH)
    train_model(build_tcn, "TCN", SEQ_LENGTH)



===== Running CNN-LSTM =====
Epoch 1/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - accuracy: 0.7280 - auc_3: 0.8020 - loss: 0.6581 - precision_3: 0.8271 - recall_3: 0.6962 - val_accuracy: 0.7645 - val_auc_3: 0.7323 - val_loss: 0.6642 - val_precision_3: 0.2903 - val_recall_3: 0.7105 - learning_rate: 1.0000e-04
Epoch 2/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8092 - auc_3: 0.8555 - loss: 0.5670 - precision_3: 0.8186 - recall_3: 0.8797 - val_accuracy: 0.8410 - val_auc_3: 0.7275 - val_loss: 0.5708 - val_precision_3: 0.3871 - val_recall_3: 0.6316 - learning_rate: 1.0000e-04
Epoch 3/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8184 - auc_3: 0.8594 - loss: 0.4579 - precision_3: 0.8409 - recall_3: 0.8633 - val_accuracy: 0.9235 - val_auc_3: 0.7654 - val_loss: 0.3913 - val_precision_3: 0.6757 - val_recall_3: 0.6579 - learning_rate: 1.0000e-04
Epoch 4/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8046 - auc_3: 0.8774 - loss: 0.4306 - precision_3: 0.8461 - r